In [13]:
# Importing several libraries
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6.0, 4.0) #setting figure size
import seaborn as sns
import matplotlib.dates as mdates
import os
import emoji
import re
pd.set_option('display.float_format', '{:.2f}'.format) # Set the float display option

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /Users/lgfolder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
DATAFILE = '../data/merged_data.csv'
df_all = pd.read_csv(DATAFILE)
df = df_all.copy().head(200)

/var/folders/j6/lnskfknd3zz28plgqdh6w_4m0000gn/T/ipykernel_92043/3170336875.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(DATAFILE)


In [143]:
i = 30
print(df["selftext"].loc[i])
post = df["selftext"].loc[i]

Hi, I have 10 4/16 GME $24 calls that are up 264% right now.

My question is: Would it be smart to hold onto these calls as GME continues to rise, or sell a portion and buy further OTM calls with a shorter expiry (thinking 1/29 $55c)? My line of thinking is that having more calls = more $. After reading the gamma squeeze DD it seems certain that GME could reach well over $55 by next week.

Appreciate all and any advice as I'm still learning everyday. Thank you.


In [145]:
# Calculate the sentiment scores for the entire block of text
# post = ["Hey guys. So I just saw yesterday some GME posts with a lot of super optimistic in them, and since this was my first investment, I decided to buy some shares, 2500$ worth, about 125. Should I bother trading them when I think they're high, then buy back when they go down, or should I just sit back, relax and keep riding to the moon? (I'd currently sell for 800$ profit I guess) What are you guys doing right now"]

overall_sentiment_score = analyzer.polarity_scores(post)
print(overall_sentiment_score)

{'neg': 0.0, 'neu': 0.864, 'pos': 0.136, 'compound': 0.8807}


In [96]:
# sentences = sent_tokenize(post)

In [83]:
for sentence in sentences:
    sentiment_score = analyzer.polarity_scores(sentence)
    print(sentence)
    print(sentiment_score)

Guysss... we retards have fantasized a long time about this but it is finally happening!!!
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
GME confirmed Cohens role.
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
We gonna moon retards, i hope retards can breath in space.
{'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'compound': 0.4404}
[:newspaper:  |  **GameStop Announces Additional Board Refreshment to Accelerate Transformation**  https://news.gamestop.com/news-releases/news-release-details/gamestop-announces-additional-board-refreshment-accelerate](https://news.gamestop.com/news-releases/news-release-details/gamestop-announces-additional-board-refreshment-accelerate)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [ ]:
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):

	# Create a SentimentIntensityAnalyzer object.
	sid_obj = SentimentIntensityAnalyzer()

	# polarity_scores method of SentimentIntensityAnalyzer
	# object gives a sentiment dictionary.
	# which contains pos, neg, neu, and compound scores.
	sentiment_dict = sid_obj.polarity_scores(sentence)
	
	print("Overall sentiment dictionary is : ", sentiment_dict)
	print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
	print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
	print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")

	print("Sentence Overall Rated As", end = " ")

	# decide sentiment as positive, negative and neutral
	if sentiment_dict['compound'] >= 0.05 :
		print("Positive")

	elif sentiment_dict['compound'] <= - 0.05 :
		print("Negative")

	else :
		print("Neutral")



# Driver code
if __name__ == "__main__" :

	print("\n1st statement :")
	sentence = "Geeks For Geeks is the best portal for \
				the computer science engineering students."

	# function calling
	sentiment_scores(sentence)

	print("\n2nd Statement :")
	sentence = "study is going on as usual"
	sentiment_scores(sentence)

	print("\n3rd Statement :")
	sentence = "I am very sad today."
	sentiment_scores(sentence)


In [147]:
# code to extract stock ticker mentions from the social media posts 

import pandas as pd
import re

# Sample text containing stock mentions
text = "I just bought $GME and $AAPL, and I'm considering $AMZN as well. $gme is on fire! GME, Gamestop, gamestop, GameStop, gme."

# Sample list of publicly traded tickers
ticker_data = {
    'Ticker': ['$GME', '$AAPL', '$AMZN', '$TSLA'],
    'Company': ['GameStop Inc.', 'Apple Inc.', 'Amazon.com Inc.', 'Tesla, Inc.']
}

# Create a DataFrame from the ticker data
ticker_df = pd.DataFrame(ticker_data)

# Define a regular expression pattern to find stock mentions
pattern = r'\$?(?i)\b(' + '|'.join(map(re.escape, ticker_df['Ticker'])) + r')\b'

# Find all stock mentions in the text
mentions = re.findall(pattern, text)

# Remove duplicate mentions
unique_mentions = list(set(mentions))

# Create a dictionary to store the mentions and associated companies
mentions_dict = {}
for mention in unique_mentions:
    company = ticker_df[ticker_df['Ticker'].str.lower() == mention.lstrip('$').lower()]['Company'].values[0]
    mentions_dict[mention] = company

# Create a DataFrame from the mentions dictionary
mentions_df = pd.DataFrame.from_dict(mentions_dict, orient='index', columns=['Company'])

# Print the extracted mentions
print(mentions_df)


Empty DataFrame
Columns: [Company]
Index: []
